In [4]:
from datetime import datetime as dt

#######################
# standard code block #
#######################

# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

#######################
#       imports       #
#######################
import pandas as pd
import numpy as np
import seaborn as sns

import itertools
import pickle


from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.datasets.california_housing import fetch_california_housing

from sklearn.ensemble import GradientBoostingClassifier

# change margin size of jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

sns.set_style("whitegrid")

Populating the interactive namespace from numpy and matplotlib


variables to shift
* how many weeks to rollon 
* 

stuff to visualize 
* earnings over time as a subsection of total holdings 

In [5]:
btcusd = pd.read_csv('btcusd.csv')
ltcusd = pd.read_csv('ltcusd.csv')
ethusd = pd.read_csv('ethusd.csv')
print(btcusd.shape,ltcusd.shape, ethusd.shape)
crypto = pd.concat([ethusd,ltcusd,btcusd])

(1451, 8) (320, 8) (1237, 8)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [21]:
ethusd

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
0,2019-09-27,ETHUSD,167.46,167.46,165.85,165.85,0.00,0.00
1,2019-09-26,ETHUSD,169.87,170.93,152.11,167.46,26025.31,4231157.10
2,2019-09-25,ETHUSD,166.15,175.00,162.47,169.87,16209.66,2728669.98
3,2019-09-24,ETHUSD,200.68,202.69,152.39,166.15,57331.20,9978628.61
4,2019-09-23,ETHUSD,211.37,211.41,198.34,200.68,11158.06,2298918.33
...,...,...,...,...,...,...,...,...
1232,2016-05-13,ETHUSD,10.20,11.59,10.20,10.69,1769.71,18923.55
1233,2016-05-12,ETHUSD,10.43,12.00,9.92,10.20,2072.56,22183.39
1234,2016-05-11,ETHUSD,9.68,10.47,9.68,10.43,3052.51,30978.11
1235,2016-05-10,ETHUSD,9.98,9.98,9.36,9.68,672.06,6578.20


Given that you can only hold one type of crypto at any time, and assuming you start with $1,000 USD, analyze the historical price data provided for BTC, ETH, and LTC: on which days would you trade from one currency to another in order to maximize returns? Assume your trades are filled at the average daily price for each coin on the days you trade. What is your trade history and ending account balance between Jan 01, 2018 and Sept 01, 2019?

# base model 1

for each end of week day (monday), find the average direction of all crypto from the previous week. 
Based on overall direction shift to the crypto with the historically most traction up, or the least traction down. 
the proportion of the shift is relative to the intensity of the shift. 

steps)
1) create week #

2) collapse onto week

3) find week min and max and take diff as growth

4) join back onto week

5) single out end of week days, 

6) find the historical sensitivity up and down for each coin doing shifts

6a) as a correlation of each coin relative to the average change in the overall market. 

7) trade at end of week prices based on average direction * highest sensitivity when going up or lowest sensitivity when going down


# base model 2

will focus on trading on fridays, because a model should trade as regularly as it's model is sophisticated. 




In [22]:
ltcusd.tail()

,Date,Symbol,Open,High,Low,Close,Volume LTC,Volume USD
315,2018-11-16,LTCUSD,43.20,43.61,41.00,41.72,2287.93,96738.05
316,2018-11-15,LTCUSD,42.72,43.73,39.59,43.20,8301.32,353446.78
317,2018-11-14,LTCUSD,49.20,49.90,40.24,42.72,12849.72,551213.20
318,2018-11-13,LTCUSD,50.50,50.50,48.35,49.20,4842.46,239937.06
319,2018-11-12,LTCUSD,50.32,50.50,50.00,50.50,361.87,18176.56


In [6]:
class BuildModelData:
    
    def __init__(self, data, initialAmmount = 1000, aggBin=3, window=4):
        self.initialAmmount = initialAmmount
        self.data = data
        self.mergedCrypto = self.mergeCrypto()
        self.AggData = self.buildAggData(aggBin)
        self.buildRollingAverages(window) 
        self.buildLabel()
    
    def mergeCrypto(self):
        mergedCrypto = self.data['BTC'].merge(self.data['ETH'], how='outer', on='Date',suffixes=('_BTC', '_ETH'))
        self.data['LTC'].columns = ['Date','Symbol_LTC','Open_LTC', 'High_LTC', 'Low_LTC', 'Close_LTC', 'Volume LTC', 'Volume USD_LTC']
        mergedCrypto = mergedCrypto.merge(self.data['LTC'], how='outer', on='Date')
        mergedCrypto.drop(columns=['Symbol_BTC','Symbol_ETH','Symbol_LTC'], inplace=True)
        return mergedCrypto

    def buildAggData(self, aggBin):
        self.mergedCrypto = self.mergedCrypto.reindex(index=self.mergedCrypto.index[::-1])
        self.mergedCrypto.reset_index(inplace=True,drop=True)
        def collapseWithGrowth(cryptoData):
        # group by id/3
            #get first day      
            columns = [x for x in cryptoData.columns]
            groupByObject = cryptoData.groupby(cryptoData.index//aggBin)[columns]
            crypto_Agg_first = groupByObject.first()
            crypto_Agg_first.columns = [x +'_fstd' for x in columns]
            crypto_Agg_first.reset_index(inplace=True)

            crypto_Agg_last = groupByObject.last()
            crypto_Agg_last.columns = [x +'_lstd' for x in columns]
            crypto_Agg_last.reset_index(inplace=True)
            crypto_Agg = crypto_Agg_first.merge(crypto_Agg_last, how='outer', on='index')
            

        # findGrowth
            for eachCrypto in self.data:
                
                #get price growth 
                crypto_Agg['growth_'+eachCrypto] = (crypto_Agg['price_'+eachCrypto+'_lstd'] - crypto_Agg['price_'+eachCrypto+'_fstd'])/crypto_Agg['price_'+eachCrypto+'_fstd']        
                #get volume change Volume LTC_lstd
                crypto_Agg['volu_growth_'+eachCrypto] = (crypto_Agg['Volume '+eachCrypto+'_lstd'] - crypto_Agg['Volume '+eachCrypto+'_fstd'])/crypto_Agg['Volume '+eachCrypto+'_fstd']        
                #get price volatility
                crypto_Vol = cryptoData.groupby(cryptoData.index//aggBin)['price_'+eachCrypto].agg({ 'volatility_'+eachCrypto : np.std })
                crypto_Agg['volat_'+eachCrypto] = crypto_Vol['volatility_'+eachCrypto]
                #get price volatility
                
            dropcolumns = [x for x in crypto_Agg.columns if x.startswith(('Open','High','Close','Low'))]
            crypto_Agg.drop(dropcolumns, axis=1, inplace=True)
            return crypto_Agg
        

        #get average price per day
        for eachCrypto in self.data:
            self.mergedCrypto['price_'+eachCrypto] = (self.mergedCrypto['High_'+eachCrypto] + self.mergedCrypto['Low_'+eachCrypto])/2
            self.mergedCrypto['range_'+eachCrypto] = self.mergedCrypto['High_'+eachCrypto] - self.mergedCrypto['Low_'+eachCrypto]
        
        return collapseWithGrowth(self.mergedCrypto)
                        
    def buildLabel(self):
        def whichCrypto(row, direction):
            b = row['growth_BTC']
            e = row['growth_ETH']
            l = row['growth_LTC']

            maxGrowthForThatDay = np.nanmax([b,e,l])
            minGrowthForThatDAy = np.nanmin([b,e,l])
            
            if direction == 'up':
                comp = maxGrowthForThatDay # what will gain you the most money
            elif direction == 'dwn':
                comp = minGrowthForThatDAy #what will lose you the most money

            if comp == b:
                return 'BTC'
            elif comp == e:
                return 'ETH'
            elif comp == l:
                return 'LTC'

        self.AggData['WhichCryptoUp'] = self.AggData.apply(whichCrypto, direction='up', axis=1)
        self.AggData['WhichCryptoDwn'] = self.AggData.apply(whichCrypto, direction='dwn', axis=1)
        self.AggData['upORdwn'] = self.AggData[['growth_BTC','growth_ETH','growth_LTC']].mean(axis=1)
        self.AggData['upORdwn'] = self.AggData['upORdwn'].apply(lambda x: 1 if x > 0 else 0)
    
    
    def buildRollingAverages(self,window):
        self.AggData.sort_index(inplace=True)
        for i in range(len(self.AggData.columns)):
            name = self.AggData.columns[i]
            dtype = self.AggData[name].dtype
            if dtype =='float':
                self.AggData['rol_'+ name] = self.AggData.iloc[:,i].rolling(window=window).mean()
        
    
A1 = BuildModelData({'BTC': btcusd,'ETH':ethusd,'LTC':ltcusd})


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)



# Train Model on Prior Period

## build label

## separate out features

In [44]:
AggData = A1.AggData

In [28]:
AggData

,index,Date_fstd,Volume BTC_fstd,Volume USD_BTC_fstd,Volume ETH_fstd,Volume USD_ETH_fstd,Volume LTC_fstd,Volume USD_LTC_fstd,price_BTC_fstd,range_BTC_fstd,...,rol_volat_BTC,rol_growth_ETH,rol_volu_growth_ETH,rol_volat_ETH,rol_growth_LTC,rol_volu_growth_LTC,rol_volat_LTC,WhichCryptoUp,WhichCryptoDwn,upORdwn
0,0,2015-10-08,18.80,4595.84,NaN,NaN,NaN,NaN,243.750,2.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BTC,BTC,1
1,1,2015-10-11,12.22,3021.12,NaN,NaN,NaN,NaN,247.900,3.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BTC,BTC,1
2,2,2015-10-14,119.54,30115.77,NaN,NaN,NaN,NaN,252.375,4.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BTC,BTC,1
3,3,2015-10-17,664.66,180981.35,NaN,NaN,NaN,NaN,274.070,21.86,...,2.536863,NaN,NaN,NaN,NaN,NaN,NaN,BTC,BTC,0
4,4,2015-10-20,202.09,54949.01,NaN,NaN,NaN,NaN,271.005,9.97,...,2.812039,NaN,NaN,NaN,NaN,NaN,NaN,BTC,BTC,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,479,2019-09-14,288.31,2986871.05,5842.43,1087778.85,2573.39,180178.57,10338.030,211.98,...,92.089386,0.015369,0.275755,1.841889,0.013027,0.494636,0.847442,ETH,BTC,1
480,480,2019-09-17,661.45,6760798.55,16399.13,3363033.45,12330.89,919460.44,10211.320,143.18,...,102.656860,0.023871,0.215677,2.501087,0.006620,0.692801,0.895348,ETH,BTC,1
481,481,2019-09-20,925.30,9402772.44,7633.77,1655313.62,8190.57,614659.86,10166.225,217.35,...,95.730606,0.017515,0.203111,3.356752,-0.006527,0.510749,0.989623,BTC,LTC,0
482,482,2019-09-23,2114.59,20720560.84,11158.06,2298918.33,25770.89,1737183.76,9831.790,432.08,...,240.536730,-0.028759,0.478111,7.858648,-0.045652,0.340630,2.327220,BTC,ETH,0


In [26]:
jointDataFrame.tail()['WhichCryptoDwn']

NameError: name 'jointDataFrame' is not defined

In [43]:
AggData = A1.AggData
class CryptoModel:
    
    initial_breakdown = {'BTC': 0.030638125273752783, 'ETH': 0.80073259360567473, 'LTC': 0}

    def __init__(self, date, prev_holdings, ifusd_crypto_prev, ifusd_USD_prev_val, past_data, curr_data): 
        self.date = date
        self.b_price = curr_data['price_BTC_fstd']
        self.e_price = curr_data['price_ETH_fstd'] if not math.isnan(curr_data['price_ETH_fstd']) else 0
        self.l_price = curr_data['price_LTC_fstd'] if not math.isnan(curr_data['price_LTC_fstd']) else 0
        #crypto holdings only
        self.prev_holdings = prev_holdings
        self.prev_holdings_val = self.convertHoldingsToUSD(self.prev_holdings)
        self.init_holdings_val = self.convertHoldingsToUSD(self.initial_breakdown)
        
        # ifusd vars
        self.ifusd_crypto_prev = ifusd_crypto_prev
        self.ifusd_crypto_prev_val = self.convertHoldingsToUSD(ifusd_crypto_prev)
        self.ifusd_USD_prev_val = ifusd_USD_prev_val
        
        self.past_data = past_data
        self.curr_data = curr_data
        
        self.model_up = self.buildCoinModel('WhichCryptoUp')
        self.model_dwn = self.buildCoinModel('WhichCryptoDwn')
        self.model_dir = self.buildDirModel()
        self.predictedupORdwn = self.predictDir()
        self.predictions_Up = self.setPredictions(self.model_up)
        self.predictions_Dwn = self.setPredictions(self.model_dwn)
        self.find_holdingChanges()
    
    def buildDirModel(self):
        #label
        y = self.past_data['upORdwn']
        # features
        X = self.past_data.filter(regex='^rol', axis=1).fillna(-99999999)
        X['date'] = X.index
        X['date'] = X['date'].apply(lambda x: x.toordinal())
        #fit
        dir_model = GradientBoostingClassifier(n_estimators=10)
        dir_model.fit(X, y)
        #score
        self.dir_R2 = np.round(dir_model.score(X, y), 4)
#         print(f'dir model R^2 Score: {np.round(dir_model.score(X, y), 4)}')
        
        return dir_model
        
    def buildCoinModel(self,which_crypto_upORdwn):
        # label
        cleanup_nums = {'BTC': 0,"ETH": 1, 'LTC': 2} #this should be whichCrypto
        y = self.past_data[which_crypto_upORdwn].replace(cleanup_nums).astype(int)
        # features
        X = self.past_data.filter(regex='^rol', axis=1).fillna(-99999999)
        X['date'] = X.index
        X['date'] = X['date'].apply(lambda x: x.toordinal())
        #fit
        coin_model = GradientBoostingClassifier(n_estimators=10)
        coin_model.fit(X, y)
        #score 
        self.coin_R2 = np.round(coin_model.score(X, y), 4)
#         print(f'coin model R^2 Score: {np.round(coin_model.score(X, y), 4)}')
        
#         def bias_metric(y_true, y_pred):
#             return np.mean((np.mean(y_pred) - y_true)**2)

#         def variance_metric(y_true, y_pred):
#             return np.var(y_true - y_pred)
        
#         self.bias = bias_metric(y_true, y_pred)
#         self.variance = variance_metric(y_true, y_pred)

        return coin_model

    def predictDir(self):
        date=self.curr_data['date'].toordinal()
        X = self.curr_data.filter(regex='^rol').fillna(-99999999)       
        X['date'] = date
        self.dir_probs_soft = self.model_dir.predict_proba( X.values.reshape(1, -1))
        return self.model_dir.predict(X.values.reshape(1, -1))
    
    def setPredictions(self, model):
        date=self.curr_data['date'].toordinal()
        X = self.curr_data.filter(regex='^rol').fillna(-99999999)       
        X['date'] = date
        probs_ = model.predict_proba( X.values.reshape(1, -1))
        probs = {}
        
        probs['BTC'] = probs_[0][0]
        if len(probs_[0])>1:
            probs['ETH'] = probs_[0][1]
        if len(probs_[0])>2:   
            probs['LTC'] = probs_[0][0]
        return probs
        
    def convertHoldingsToUSD(self,holdings):
        return holdings['BTC'] * self.b_price + holdings['ETH'] * self.e_price + holdings['LTC'] * self.l_price
            
    def find_holdingChanges(self):
        dwn_inverse = {x: 1-self.predictions_Dwn[x] for x in self.predictions_Dwn}
        predictions = self.predictions_Up if self.predictedupORdwn == 1 else dwn_inverse         
        prob_upORdwn = self.dir_probs_soft[0][1] if self.predictions_Up else self.dir_probs_soft[0][0]
        
        relative_predictions = {}
        total_confidence = math.fsum(predictions.values()) # sum up all
        relative_predictions['BTC'] = predictions['BTC'] / total_confidence
        if len(predictions) > 1: relative_predictions['ETH'] = predictions['ETH'] / total_confidence
        if len(predictions) > 2: relative_predictions['LTC'] = predictions['LTC'] / total_confidence
        
        amountToChange_portion =  total_confidence * self.dir_R2 *prob_upORdwn
        amountToChange = self.prev_holdings_val * amountToChange_portion
        amountRemains = self.prev_holdings_val - amountToChange
        
        def determinHoldover(amountRemains, prev_holdings, prev_holdings_val):
            BTCShare = (prev_holdings['BTC' ] * self.b_price) / prev_holdings_val
            ETHShare = (prev_holdings['ETH' ] * self.e_price) / prev_holdings_val 
            LTCShare = (prev_holdings['LTC' ] * self.l_price) / prev_holdings_val
            
            holdOver_BTC = (amountRemains * BTCShare)/ self.b_price
            holdOver_ETH = (amountRemains * ETHShare)/ self.e_price
            holdOver_ETH = holdOver_ETH if not math.isnan(holdOver_ETH) else 0.0

            if self.l_price == 0 or math.isnan(self.l_price):
                holdOver_LTC = 0
            else:
                holdOver_LTC = (amountRemains * LTCShare )/ self.l_price
                
            return [holdOver_BTC,  holdOver_ETH,holdOver_LTC]
        
#         holdOver_BTC, holdOver_ETH, holdOver_LTC = determinHoldover(amountRemains, self.prev_holdings, self.prev_holdings_val)
                
        def findChanges(relative_predictions,amountToChange):
            changes_BTC = amountToChange*relative_predictions['BTC'] / self.b_price
            changes_ETH = amountToChange*relative_predictions['ETH'] / self.e_price if 'ETH' in predictions else 0
            changes_LTC = amountToChange*relative_predictions['LTC'] / self.l_price if 'LTC' in predictions else 0
            return [changes_BTC, changes_ETH, changes_LTC]
        
        h = determinHoldover(amountRemains, self.prev_holdings, self.prev_holdings_val)
        c = findChanges(relative_predictions, amountToChange)
        
        def setHoldings(h, c):
            new_holdings = {}
            new_holdings['BTC'] = h[0] + c[0]
            new_holdings['ETH'] = h[1] + c[1]
            new_holdings['LTC'] = h[2] + c[2]
            
            new_holdings_val = self.convertHoldingsToUSD(new_holdings)
            
            new_holdings_rel = {}    
            new_holdings_rel['BTC'] = ((h[0] + c[0])*self.b_price)/new_holdings_val
            new_holdings_rel['ETH'] = ((h[1] + c[1])*self.e_price)/new_holdings_val
            new_holdings_rel['LTC'] = ((h[2] + c[2])*self.l_price)/new_holdings_val
        
            return new_holdings, new_holdings_rel, new_holdings_val
        
        self.new_holdings, self.new_holdings_rel, self.new_holdings_val = setHoldings(h, c)
        
        def determine_earnings_ifyoucantradeintoUSD(predictedupORdwn, relative_predictions, findChanges):
            
            if predictedupORdwn == 1: # buy into crypto
                cryptoToBuy = self.ifusd_USD_prev_val * amountToChange_portion
                print("to buy: ", cryptoToBuy,"ifusd_cash_val: ", self.ifusd_USD_prev_val, "prev crypto val: ", self.ifusd_crypto_prev_val,"% to change: ", amountToChange_portion)
                cryptoToChange = (self.ifusd_crypto_prev_val-cryptoToBuy) * amountToChange_portion
                cryptoChanges = cryptoToChange + cryptoToBuy
                cryptoRemains = self.ifusd_crypto_prev_val - cryptoToChange
                
                self.ifusd_USD_new_val = self.ifusd_USD_prev_val - cryptoToBuy
                
                h = determinHoldover(cryptoRemains, self.ifusd_crypto_prev, self.ifusd_crypto_prev_val)
                c = findChanges(relative_predictions, cryptoChanges)

                self.ifusd_crypto_new, self.ifusd_crypto_new_rel,self.ifusd_crypto_new_val = setHoldings(h, c)
                
            else: 
                # sell out of crypto
                crypto_amountToSell = self.ifusd_crypto_prev_val * amountToChange_portion #
#                 print("to sell: ", crypto_amountToSell, "ifusd_cash_val: ", self.ifusd_USD_prev_val , "prev crypto val: ", self.ifusd_crypto_prev_val,"% to change: ", amountToChange_portion)
                crypto_amountRemains = self.ifusd_crypto_prev_val - crypto_amountToSell 
                self.ifusd_USD_new_val = self.ifusd_USD_prev_val +  crypto_amountToSell
                
                # change up existing crypto
                crypto_amountToChange = crypto_amountRemains * amountToChange_portion 
                crypto_amountNotToChange = crypto_amountRemains - crypto_amountToChange
                
                h = determinHoldover(crypto_amountNotToChange, self.ifusd_crypto_prev, self.ifusd_crypto_prev_val)
                c = findChanges(relative_predictions, crypto_amountToChange)
                
                self.ifusd_crypto_new, self.ifusd_crypto_new_rel, self.ifusd_crypto_new_val = setHoldings(h, c)         
            
            self.ifusd_total_bal_new = self.ifusd_crypto_new_val + self.ifusd_USD_new_val
            print(self.new_holdings_val,'noChangeVal')
            print(self.ifusd_total_bal_new,'withUSD_newBal', 'USD', self.ifusd_USD_new_val, 'crypto', self.ifusd_crypto_new_val)
#             print(self.init_holdings_val,'initial')           

        determine_earnings_ifyoucantradeintoUSD(self.predictedupORdwn,relative_predictions,findChanges)
        
def setup(initial_amount,AggData,startDate, endDate,):
    startdate_DT = dt.strptime(startDate, '%Y-%m-%d').date()
    AggData['date'] = pd.to_datetime(AggData['Date_fstd']) 
    AggData.set_index('date',inplace=True, drop=False)
    dataBefore = AggData[pd.to_datetime(AggData['Date_fstd']) < startdate_DT]
    
    def find_intial_holdings(startDate):
        b_mean = dataBefore['growth_BTC'].mean()
        e_mean = dataBefore['growth_ETH'].mean()
        b_share = b_mean/(b_mean+e_mean)
        e_share = e_mean/(b_mean+e_mean)
        
        b_price = AggData.loc[startDate]['price_BTC_fstd']
        e_price = AggData.loc[startDate]['price_ETH_fstd']

        initial_holdings = {
            'BTC': ((initial_amount*b_share)/b_price),
            'ETH': ((initial_amount*e_share)/e_price),
            'LTC': 0
        }

        return initial_holdings
        
    return CryptoModel(date=startDate, prev_holdings=find_intial_holdings(startdate_DT), ifusd_crypto_prev=find_intial_holdings(startdate_DT),ifusd_USD_prev_val=0, past_data=dataBefore, curr_data=AggData.loc[startdate_DT] )

first_day = setup(initial_amount=1000, startDate='2018-01-01', endDate = '2019-09-01',AggData=AggData)
 
        
#build model print('predictions', predictions, self.predictedupORdwn, self.predictions_Dwn, self.predictions_Up )

to buy:  0.0 ifusd_cash_val:  0 prev crypto val:  1000.0 % to change:  0.4367367311046875
1000.0 noChangeVal
1000.0 withUSD_newBal USD 0.0 crypto 1000.0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:207: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.


In [35]:
print(0.3115953681716913+ 0.7177804653132523)
# first_day
startDate_2 = '2018-01-04'
startdate_2 = dt.strptime(startDate_2, '%Y-%m-%d').date()
dataBefore = AggData[(pd.to_datetime(AggData['Date_fstd']) < startdate_2)]
second_day = CryptoModel(date=startDate_2, prev_holdings=first_day.new_holdings, ifusd_crypto_prev=first_day.ifusd_crypto_new, ifusd_USD_prev_val=first_day.ifusd_USD_new_val, past_data=dataBefore, curr_data=AggData.loc[startDate_2])


# # printout(second_day)

# # Plot feature importance
# feature_importance = second_day.feature_importances_
# # make importances relative to max importance
# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# # sorted_idx = np.argsort(feature_importance)
# pos = np.arange(sorted_idx.shape[0]) + .5
# plt.subplot(1, 2, 2)
# plt.barh(pos, feature_importance[sorted_idx], align='center')
# # plt.yticks(pos, boston.feature_names[sorted_idx])
# plt.xlabel('Relative Importance')
# plt.title('Variable Importance')
# plt.show()


1.0293758334849437
to buy:  0.0 ifusd_cash_val:  0.0 prev val:  1206.1172356626114 % to change:  0.5433858522087807
1206.1172356626114 noChangeVal
1206.1172356626114 withUSD_newBal
1214.2647723353223 initial


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """


1) build an object with the following 

2) model assumes only prior knowledge to replicate actual trading circumstances 

3) show how the model improves over time 

4) build a bar shifter to show how these changes happen over time, 

[date, B_bal, E_bal, L_bal, total_holdings, prev_date, model_built_toDate, prev_period, predicted_direction, predicted_coinIfUp, predicted_coinIfDwn ]

## issues 
1) why does the previous value not match


In [44]:
startdate = dt.strptime('2018-01-01', '%Y-%m-%d').date()
dashboard_data = {startdate: first_day}

dateArray = [first_day.date]
holdings_rel = [first_day.new_holdings_rel]
account_value = [first_day.new_holdings_val]
valueof_intial_holding = [first_day.init_holdings_val]
models = []
for i in range(AggData[AggData['date'] < startdate].shape[0]+1, AggData.shape[0] ):    
    date = AggData.iloc[i]['date'].date()
    dateBefore = AggData.iloc[i-1]['date'].date()
    dataBefore = AggData[AggData['date'] < pd.Timestamp(date)]
    
    prevModel = dashboard_data[dateBefore]
    prev_holdings = prevModel.new_holdings
    print(date)
    model = CryptoModel(date=date, prev_holdings=prev_holdings, ifusd_crypto_prev=prevModel.ifusd_crypto_new, ifusd_USD_prev_val=prevModel.ifusd_USD_new_val, past_data=dataBefore, curr_data=AggData.loc[date])
        
    df = AggData.loc[date]
#     print('outcomes actual: ', df.upORdwn, 'ifUp: ', df.WhichCryptoUp, 'ifDwn_', df.WhichCryptoDwn )
#     print('outcomes predicted for next: ', model.predictedupORdwn, 'ifUp: ', model.predictions_Up, 'ifDwn_', model.predictions_Dwn  )
    
    dashboard_data[date] = model 
    models.append(model)
    dateArray.append(model.date)
    holdings_rel.append(model.new_holdings_rel)
    account_value.append(model.new_holdings_val)
    valueof_intial_holding.append(model.init_holdings_val)

    
    
    
# [date, B_bal, E_bal, L_bal, total_holdings, prev_date, model_built_toDate, prev_period, predicted_direction, predicted_coinIfUp, predicted_coinIfDwn ]


# 1) make sure it's going in the direction you expect 
# 2) make sure it's buying into the coins you expect 

#834


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  if __name__ == '__main__':


2018-01-04
to buy:  0.0 ifusd_cash_val:  0.0 prev crypto val:  1206.1172356626114 % to change:  0.5433858522087807
1206.1172356626114 noChangeVal
1206.1172356626114 withUSD_newBal USD 0.0 crypto 1206.1172356626114
2018-01-07
to buy:  0.0 ifusd_cash_val:  0.0 prev crypto val:  1342.6377766856124 % to change:  0.6379298312019133
1342.6377766856124 noChangeVal
1342.6377766856124 withUSD_newBal USD 0.0 crypto 1342.6377766856124
2018-01-10
to buy:  0.0 ifusd_cash_val:  0.0 prev crypto val:  1423.731458992168 % to change:  0.6528774639977681
1423.731458992168 noChangeVal
1423.731458992168 withUSD_newBal USD 0.0 crypto 1423.731458992168
2018-01-13
to buy:  0.0 ifusd_cash_val:  0.0 prev crypto val:  1431.910217271425 % to change:  0.44223596683778776
1431.9102172714247 noChangeVal
1431.9102172714247 withUSD_newBal USD 0.0 crypto 1431.9102172714247
2018-01-16
to sell:  355.3099142381063 ifusd_cash_val:  0.0 prev crypto val:  1182.275655167471 % to change:  0.3005305172995177
1182.2756551674706 

to buy:  11.474497207365884 ifusd_cash_val:  19.048687265149102 prev crypto val:  1114.4884835893736 % to change:  0.6023773212109621
747.857302653267 noChangeVal
1133.5371708545226 withUSD_newBal USD 7.574190057783218 crypto 1125.9629807967394
2018-04-22
to buy:  4.612924245312161 ifusd_cash_val:  7.574190057783218 prev crypto val:  1247.0703444763753 % to change:  0.6090320166407671
828.292871081016 noChangeVal
1254.6445345341585 withUSD_newBal USD 2.9612658124710567 crypto 1251.6832687216875
2018-04-25
to buy:  1.8471764584547035 ifusd_cash_val:  2.9612658124710567 prev crypto val:  1310.7686613806513 % to change:  0.6237793482353107
867.391951704953 noChangeVal
1313.7299271931224 withUSD_newBal USD 1.1140893540163532 crypto 1312.615837839106
2018-04-28
to buy:  0.7042339186426669 ifusd_cash_val:  1.1140893540163532 prev crypto val:  1323.2556081838559 % to change:  0.6321161907739851
874.4230333878843 noChangeVal
1324.369697537872 withUSD_newBal USD 0.4098554353736863 crypto 1323.9

to buy:  6.626421896693034 ifusd_cash_val:  12.74642749567725 prev crypto val:  1314.5906900531004 % to change:  0.5198650287651407
713.6402757845715 noChangeVal
1327.3371175487775 withUSD_newBal USD 6.120005598984215 crypto 1321.2171119497934
2018-07-30
to buy:  2.7131875647563435 ifusd_cash_val:  6.120005598984215 prev crypto val:  1316.7062635079308 % to change:  0.44333089584210056
711.2014560000823 noChangeVal
1322.8262691069149 withUSD_newBal USD 3.4068180342278716 crypto 1319.419451072687
2018-08-02
to buy:  1.3687302160473582 ifusd_cash_val:  3.4068180342278716 prev crypto val:  1226.0820843731938 % to change:  0.4017620554710872
660.8880795203984 noChangeVal
1229.4889024074214 withUSD_newBal USD 2.0380878181805135 crypto 1227.450814589241
2018-08-05
to sell:  397.5927449764097 ifusd_cash_val:  2.0380878181805135 prev crypto val:  1156.5287759687162 % to change:  0.34378110881277757
622.7038482142414 noChangeVal
1158.5668637868966 withUSD_newBal USD 399.63083279459016 crypto 75

to buy:  11.51296762480177 ifusd_cash_val:  25.14061859687882 prev crypto val:  996.1447650660725 % to change:  0.4579428935066495
450.93301562703766 noChangeVal
1021.2853836629513 withUSD_newBal USD 13.627650972077051 crypto 1007.6577326908742
2018-11-06
to buy:  6.6246326675144545 ifusd_cash_val:  13.627650972077051 prev crypto val:  1045.5701680643663 % to change:  0.4861169897209926
467.90626747628426 noChangeVal
1059.1978190364432 withUSD_newBal USD 7.003018304562596 crypto 1052.1948007318806
2018-11-09
to buy:  3.9072945674604465 ifusd_cash_val:  7.003018304562596 prev crypto val:  1039.724202432995 % to change:  0.5579443602074796
462.3599727876773 noChangeVal
1046.7272207375577 withUSD_newBal USD 3.09572373710215 crypto 1043.6314970004555
2018-11-12
to buy:  1.4480628814495835 ifusd_cash_val:  3.09572373710215 prev crypto val:  1041.4542778715572 % to change:  0.46776230840452465
461.3955410059131 noChangeVal
1044.5500016086596 withUSD_newBal USD 1.6476608556525665 crypto 1042.

to buy:  55.88672246305711 ifusd_cash_val:  95.855842504235 prev crypto val:  667.0308131937895 % to change:  0.5830288587843562
316.94330077370944 noChangeVal
762.8866556980247 withUSD_newBal USD 39.96912004117789 crypto 722.9175356568468
2019-02-16
to sell:  376.96364893661104 ifusd_cash_val:  39.96912004117789 prev crypto val:  727.143754237423 % to change:  0.5184169522736861
318.6919537379609 noChangeVal
767.1128742786008 withUSD_newBal USD 416.9327689777889 crypto 350.1801053008119
2019-02-19
to buy:  272.50410839644263 ifusd_cash_val:  416.9327689777889 prev crypto val:  396.36959071272315 % to change:  0.6535924462463147
360.9346211887322 noChangeVal
813.302359690512 withUSD_newBal USD 144.42866058134626 crypto 668.8736991091657
2019-02-22
to buy:  111.58275116818388 ifusd_cash_val:  144.42866058134626 prev crypto val:  675.4503857386164 % to change:  0.7725803917245175
364.4698333700895 noChangeVal
819.8790463199628 withUSD_newBal USD 32.84590941316239 crypto 787.0331369068003

to buy:  312.09393835443785 ifusd_cash_val:  443.31224799366726 prev crypto val:  708.6062264356655 % to change:  0.7040047726335233
673.6268541389645 noChangeVal
1151.9184744293327 withUSD_newBal USD 131.2183096392294 crypto 1020.7001647901034
2019-05-26
to buy:  79.82163403895736 ifusd_cash_val:  131.2183096392294 prev crypto val:  1146.5623715003353 % to change:  0.6083117078585933
756.9304361959416 noChangeVal
1277.7806811395649 withUSD_newBal USD 51.39667560027205 crypto 1226.3840055392927
2019-05-29
to buy:  33.106810587950285 ifusd_cash_val:  51.39667560027205 prev crypto val:  1277.5182775476642 % to change:  0.6441430345696336
788.5225097805501 noChangeVal
1328.914953147936 withUSD_newBal USD 18.289865012321762 crypto 1310.6250881356143
2019-06-01
to buy:  13.058817533912707 ifusd_cash_val:  18.289865012321762 prev crypto val:  1306.6352972044915 % to change:  0.7139920127958882
786.1188603286619 noChangeVal
1324.925162216813 withUSD_newBal USD 5.231047478409055 crypto 1319.69

to buy:  6.4804157280250045 ifusd_cash_val:  9.428270709226599 prev crypto val:  990.6219519453684 % to change:  0.6873387419480017
602.1123935918943 noChangeVal
1000.050222654595 withUSD_newBal USD 2.947854981201594 crypto 997.1023676733935
2019-09-02
to buy:  1.5002270299038545 ifusd_cash_val:  2.947854981201594 prev crypto val:  1048.7352252723051 % to change:  0.5089215851766009
633.2914330179572 noChangeVal
1051.6830802535067 withUSD_newBal USD 1.4476279512977397 crypto 1050.2354523022088
2019-09-05
to buy:  0.986433203721307 ifusd_cash_val:  1.4476279512977397 prev crypto val:  1061.301512975353 % to change:  0.6814134825436395
639.9655840277294 noChangeVal
1062.7491409266509 withUSD_newBal USD 0.46119474757643264 crypto 1062.2879461790744
2019-09-08
to buy:  0.2930576845700105 ifusd_cash_val:  0.46119474757643264 prev crypto val:  1091.2160335948515 % to change:  0.6354315310614912
657.3934468239095 noChangeVal
1091.6772283424282 withUSD_newBal USD 0.16813706300642212 crypto 109

In [103]:
# feature_importance = models[-1].model_dir.feature_importances_[:30]
X = models[-1].past_data.filter(regex='^rol', axis=1).fillna(-99999999)

model = models[-1].model_dir
predictions = model.predict(X)
cleanup_nums = {'BTC': 0,"ETH": 1, 'LTC': 2} #this should be whichCrypto
y = models[-1].past_data[which_crypto_upORdwn].replace(cleanup_nums).astype(int)

print(predictions)
# print(featuresImp,featureNames)

# data = {'features':featuresImp, 'names':featureNames }
# df = pd.DataFrame(data)
# dfClean = df[df['features'] > 0.1]
# df_sorted= df.sort_values('features', ascending=False)

# print(df_sorted.names.values)

# # y = self.past_data[which_crypto_upORdwn].replace(cleanup_nums).astype(int)
# # # features
# # X = self.past_data.filter(regex='^rol', axis=1).fillna(-99999999)

# # # Plot feature importance
# # feature_importance = second_day.feature_importances_
# # # make importances relative to max importance
# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# sorted_idx = np.argsort(feature_importance)
# pos = np.arange(sorted_idx.shape[0]) + .5
# # plt.subplot(1, 2, 2)
# plt.barh(pos, feature_importance[sorted_idx], align='center')
# plt.yticks(pos, featureNames)
# plt.xlabel('Relative Importance')
# plt.title('Variable Importance')
# plt.figure(figsize=(100,50))
# plt.show()

ValueError: Number of features of the model must match the input. Model n_features is 34 and input n_features is 33 

In [46]:
AggData.columns

Index(['index', 'Date_fstd', 'Volume BTC_fstd', 'Volume USD_BTC_fstd',
       'Volume ETH_fstd', 'Volume USD_ETH_fstd', 'Volume LTC_fstd',
       'Volume USD_LTC_fstd', 'price_BTC_fstd', 'range_BTC_fstd',
       'price_ETH_fstd', 'range_ETH_fstd', 'price_LTC_fstd', 'range_LTC_fstd',
       'Date_lstd', 'Volume BTC_lstd', 'Volume USD_BTC_lstd',
       'Volume ETH_lstd', 'Volume USD_ETH_lstd', 'Volume LTC_lstd',
       'Volume USD_LTC_lstd', 'price_BTC_lstd', 'range_BTC_lstd',
       'price_ETH_lstd', 'range_ETH_lstd', 'price_LTC_lstd', 'range_LTC_lstd',
       'growth_BTC', 'volu_growth_BTC', 'volat_BTC', 'growth_ETH',
       'volu_growth_ETH', 'volat_ETH', 'growth_LTC', 'volu_growth_LTC',
       'volat_LTC', 'rol_Volume BTC_fstd', 'rol_Volume USD_BTC_fstd',
       'rol_Volume ETH_fstd', 'rol_Volume USD_ETH_fstd', 'rol_Volume LTC_fstd',
       'rol_Volume USD_LTC_fstd', 'rol_price_BTC_fstd', 'rol_range_BTC_fstd',
       'rol_price_ETH_fstd', 'rol_range_ETH_fstd', 'rol_price_LTC_fstd',
 

In [96]:
import plotly.express as px
import plotly.graph_objects as go

# fig, ax = plt.subplots()
# ax.set(title='earnings vs target (awards)', ylabel='earnings', xlabel='awards_proxy')
# plt.plot(ethusd['Date'], ethusd['Close'], alpha=.9);
# plt.plot(ethusd['Date'], ethusd['Close'], alpha=.9);
# plt.plot(ethusd['Date'], ethusd['Close'], alpha=.9);
# plt.savefig('img_6_earnings_against_target.png')

# date = btcusd['Date']
# data = btcusd['Close']

# # df = px.data.gapminder().query("country=='Canada'")

# data = {'date': date, 'data':data}

# df = pd.DataFrame(data)

Date_fstd = AggData['Date_fstd']
BTC = AggData['price_BTC_lstd']
UpORDwn = AggData['upORdwn']
vol = AggData['rol_Volume ETH_fstd']

# fig = px.line(AggData, x="Date_fstd", y="price_BTC_lstd", title='crypto data')


fig = go.Figure()

# fig.add_trace(go.Scatter(x=Date_fstd, y=BTC,
#                     mode='lines',
#                     name='BTC'))

# fig.add_trace(go.Scatter(x=Date_fstd, y=ETH,
#                     mode='lines',
#                     name='ETH'))
# fig.add_trace(go.Scatter(x=Date_fstd, y=LTC,
#                     mode='lines',
#                     name='LTC'))
fig.add_trace(go.Scatter(x=UpORDwn, y=vol,
                    mode='markers', name=''))

fig.show()




In [ ]:
dashboard_data

In [ ]:
def findHighestGrowthAverageToDate(self):
    PrevToJan2018_BTC = pd.DataFrame.copy(self.data[(self.data['Date'].dt.year < 2018) & (self.data['Symbol'] == 'BTCUSD')])
    PrevToJan2018_ETH = pd.DataFrame.copy(crypto[(self.data['Date'].dt.year < 2018) & (self.data['Symbol'] == 'ETHUSD')])

    eth_growth = (PrevToJan2018_ETH['Open'].head(1).values[0] - PrevToJan2018_ETH['Open'].tail(1).values[0])/PrevToJan2018_ETH['Open'].tail(1).values[0]
    btc_growth = (PrevToJan2018_BTC['Open'].head(1).values[0] - PrevToJan2018_BTC['Open'].tail(1).values[0])/PrevToJan2018_BTC['Open'].tail(1).values[0]

    eth_initalPercent = eth_growth / (eth_growth + btc_growth)
    btc_initalPercent = btc_growth / (eth_growth + btc_growth)

    eth_initialInvestment = A1.initialAmmount * eth_initalPercent
    btc_initialInvestment = A1.initialAmmount * btc_initalPercent


In [ ]:
    def setWeekID(self,cryptoData):
        cryptoData['Date'] = pd.to_datetime(cryptoData['Date'], infer_datetime_format=True)
        cryptoData['year'] = cryptoData['Date'].dt.year
        cryptoData['weekN'] = cryptoData['Date'].dt.week
        cryptoData['weekD'] = cryptoData['Date'].dt.day
        cryptoData['weekID'] = cryptoData['year'].astype(str).str.cat(cryptoData['weekN'].astype(str))
        return cryptoDatax